In [20]:
from satsearch import Search
import geojsonio
import shapely
import rasterio
import numpy as np

In [21]:
aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              139.22,
              34.87
            ],
            [
              140.80,
              34.87
            ],
            [
              140.80,
              36.10
            ],
            [
              139.22,
              36.10
            ],
            [
              139.22,
              34.87
            ]
          ]
        ]
      }
    }
  ]
}

In [11]:
start = '2019-01-01T00:00:00Z'
end = '2019-05-01T23:59:59Z'
bbox = [138.48,34.59,141.38,42.94]
min_cloud = 0
max_cloud = 100

results = (
            Search(
                bbox=bbox,
                time=f"{start}/{end}",
                query={
                    "eo:cloud_cover": {"gte": min_cloud, "lte": max_cloud},
                    "eo:sun_elevation": {"gt": 0},
                    "landsat:tier": {"eq": "T1"},
                    "collection": {"eq": "landsat-8-l1"},
                },
                sort=[
                    {"field": "datetime", "direction": "desc"}
                ]
            )
            .items()
            .geojson()
        )

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'LC81070352019117',
   'bbox': [138.82068, 34.95944, 141.4333, 37.10583],
   'geometry': {'type': 'Polygon',
    'coordinates': [[[139.34684810033124, 37.10526732840189],
      [141.4321467333038, 36.695005401364604],
      [140.90482368179292, 34.959714396231945],
      [138.82189214227859, 35.37702689672842],
      [139.34684810033124, 37.10526732840189]]]},
   'properties': {'collection': 'landsat-8-l1',
    'eo:gsd': 15,
    'eo:platform': 'landsat-8',
    'eo:instrument': 'OLI_TIRS',
    'eo:off_nadir': 0,
    'eo:bands': [{'full_width_half_max': 0.02,
      'center_wavelength': 0.44,
      'name': 'B1',
      'gsd': 30,
      'common_name': 'coastal'},
     {'full_width_half_max': 0.06,
      'center_wavelength': 0.48,
      'name': 'B2',
      'gsd': 30,
      'common_name': 'blue'},
     {'full_width_half_max': 0.06,
      'center_wavelength': 0.56,
      'name': 'B3',
      'gsd': 30,
      'common_name':

In [19]:
results['features'][0]

{'type': 'Feature',
 'id': 'LC81070352019117',
 'bbox': [138.82068, 34.95944, 141.4333, 37.10583],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[139.34684810033124, 37.10526732840189],
    [141.4321467333038, 36.695005401364604],
    [140.90482368179292, 34.959714396231945],
    [138.82189214227859, 35.37702689672842],
    [139.34684810033124, 37.10526732840189]]]},
 'properties': {'collection': 'landsat-8-l1',
  'eo:gsd': 15,
  'eo:platform': 'landsat-8',
  'eo:instrument': 'OLI_TIRS',
  'eo:off_nadir': 0,
  'eo:bands': [{'full_width_half_max': 0.02,
    'center_wavelength': 0.44,
    'name': 'B1',
    'gsd': 30,
    'common_name': 'coastal'},
   {'full_width_half_max': 0.06,
    'center_wavelength': 0.48,
    'name': 'B2',
    'gsd': 30,
    'common_name': 'blue'},
   {'full_width_half_max': 0.06,
    'center_wavelength': 0.56,
    'name': 'B3',
    'gsd': 30,
    'common_name': 'green'},
   {'full_width_half_max': 0.04,
    'center_wavelength': 0.65,
    'name': 'B4',
    'gs